# Step 2. Find the specific corrections with the LLM responses

In [1]:
import difflib
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize as encode
import re
import pandas as pd
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### A first look to the original vs corrected response

In [2]:
RESPONSES_FILE = "./responsesLatam.json"
CORRECTIONS_FILE = "./correctionsLatam.json"

df = pd.read_parquet("../data/cleaned-latam-xix.parquet")
with open(RESPONSES_FILE, "r") as f:
    r = json.load(f)

In [3]:
r['data'][0]

{'text': 'La publicacion del Oso se harà dos veces cada se mana, y constará de un pliego en cuarto ; ofreciendo à mas sus redactores, dar los gravados oportunos, siempre que loexija el asuntode que trate. Redactado por un Num. 8. TEMA del Periodico. POLITICA MILITAR. OCTAVA SESION. Abierta la sesion á las dore y un minuto de la noche , 25 de Febrero de 1845 , con asistencia de todos los Señores Representantes, se leyó y aprobó la acta de la Asamblea anterior , ménos en lo tocante à la torre del Convento de Santo Domingo, punto que quedó para ventilarse en mejor ocasion. En seguida se dió cuenta de una nota del Ejecutivo , referente à que urjía la necesidad de organizar un Ejército ; pues decia el Excmo. Decano: - "Un poder sin bayonetas vale tanto como un cero puesto á la izquierda."',
 'resp': 'La publicación del Oso se hará dos veces cada semana, y constará de un pliego en cuarto; ofreciendo además sus redactores, dar los grabados oportunos, siempre que lo exija el asunto de que trat

As observed, some specific kind of OCR errors are easy to detect, when the corrected text is basically the same as the original, but with some characters in the middle (spaces or punctuation). For example:
`se mana` → `semana`

## 1. Define the diff algorithm

In [4]:
WORDS_OF_CONTEXT = 5

def get_indexes(text, idxs, gold):
    text = text.replace("\n", " ").replace("\r", " ")
    tok = encode(text)
    wordstill = ""

    for tk in tok[:idxs[0]]:
        wordstill += tk
        assert text.startswith(wordstill), f"ERROR. Expected:\n{text}\nto start with\n{wordstill}"
        match = re.match(r'[\s]*', text[len(wordstill):])
        spaces = match.group(0) if match else ""
        wordstill += spaces

    firstidx = len(wordstill)

    for tk in tok[idxs[0]:idxs[1]+1]:
        wordstill += tk
        assert text.startswith(wordstill), f"ERROR. Expected:\n{text}\nto start with\n{wordstill}"
        match = re.match(r'[\s]*', text[len(wordstill):])
        spaces = match.group(0) if match else ""
        wordstill += spaces

    cut = wordstill[firstidx:len(wordstill)].rstrip() # this is the piece of text
    lastidx = len(cut)+firstidx
    assert cut.replace(" ", "") == gold.replace(" ", ""), f"ERROR. Indexing not correct! expected {gold} but got {cut}"
    return cut, firstidx, lastidx

def diff(text1, text2):
    sm = difflib.SequenceMatcher(None, encode(text1), encode(text2))
    added = []
    removed = []
    modified = []
    for opcode, a0, a1, b0, b1 in sm.get_opcodes():
        sa = ' '.join(sm.a[a0:a1])
        sb = ' '.join(sm.b[b0:b1])
        if opcode == 'insert':
            added.append(sb)
        elif opcode == 'delete':
            removed.append(sa)
        elif opcode == 'replace':
            context = ' '.join(sm.a[max(0,a0-WORDS_OF_CONTEXT):min(a1+WORDS_OF_CONTEXT, len(sm.a))])
            sa, start, end = get_indexes(text1, (a0, a1-1), sa)
            sb, _, _ = get_indexes(text2, (b0, b1-1), sb)
            modified.append((sa,sb,context,start,end))
        elif opcode == 'equal':
            pass
        else:
            raise RuntimeError(f"Unknown opcode {opcode}")
    return added, removed, modified

def remove_special(string):
    return re.sub(r'[^\w]', '', string)

def ocr_error(p, m):
    pws = remove_special(p).lower()
    mws = remove_special(m).lower()
    return pws == mws

def printred(text, word):
    index = 0
    while index < len(text):
        if text[index:index+len(word)] == word:
            print('\033[91m' + text[index:index+len(word)] + '\033[0m', end='')
            index += len(word)
        else:
            print(text[index], end='')
            index += 1

Usage example:

In [5]:
prv = """10 CONQUISTA sinti\u00f3 Cort\u00e9s como uno de los mayores contra- tiempos que se le podian ofrecer. Hizole retirar \u00e1 su quarto, y acudi\u00f3 con nueva irritaci\u00f3n ala de- fensa del quartel; pero se hall\u00f3 sin enemigos en quien tomar satisfacci\u00f3n de su enojo : porque al mismo instante que vieron caer \u00e1 su Rey, \u00f3 pu- dieron conocer que iba herido, se asombraron de su misma culpa, y huyendo sin saber de quien, 6 creyendo que llevaban \u00e1 las espaldas la ira de sus Dioses, corrieron \u00e1 esconderse del Cielo con aquel g\u00e9nero de confusi\u00f3n, \u00f3 fealdad espantosa que sue len dexar en el \u00e1nimo al acabarse de cometer los enormes delitos. Pas\u00f3 luego Hern\u00e1n Cort\u00e9s al quarto de Motezu ma, que volvi\u00f3 en si dentro de breve rato ; pero tan impaciente y despechado, que fu\u00e9 necesario detenerle para que no se quitase la vida. No era posible curarle, porque desviaba los medicamen tos : prorumpia en amenazas, que terminaban en gemidos : esforz\u00e1base la ira, y declinaba en pusi lanimidad : la persuasi\u00f3n le ofendia, y los consue los le irritaban : cobr\u00f3 el sentido para perder el entendimiento; y pareci\u00f3 conveniente dexarle por un rato, y dar alg\u00fan tiempo \u00e1 la consideraci\u00f3n, pa ra que se desembarazase de las primeras disonan cias de la ofensa. Qued\u00f3 encargado \u00e1 su familia, y en miserable congoja, batallando con las violen cias de su natural, y el abatimiento de su esp\u00edritu, sin aliento para intentar el castigo de los traydores"""
mod = """Dado el texto entre ```, retorna \u00fanicamente el texto corrigiendo los errores ortogr\u00e1ficos:\n\n```10 CONQUISTA sinti\u00f3 Cort\u00e9s como uno de los mayores contratiempos que se le pod\u00edan ofrecer. H\u00edzole retirar \u00e1 su cuarto, y acudi\u00f3 con nueva irritaci\u00f3n a la defensa del cuartel; pero se hall\u00f3 sin enemigos en quien tomar satisfacci\u00f3n de su enojo: porque al mismo instante que vieron caer \u00e1 su Rey, \u00f3 pudieron conocer que iba herido, se asombraron de su misma culpa, y huyendo sin saber de quien, 6 creyendo que llevaban \u00e1 las espaldas la ira de sus Dioses, corrieron \u00e1 esconderse del Cielo con aquel g\u00e9nero de confusi\u00f3n, \u00f3 fealdad espantosa que suelen dejar en el \u00e1nimo al acabarse de cometer los enormes delitos. Pas\u00f3 luego Hern\u00e1n Cort\u00e9s al cuarto de Moctezuma, que volvi\u00f3 en s\u00ed dentro de breve rato; pero tan impaciente y despechado, que fue necesario detenerle para que no se quitase la vida. No era posible curarle, porque desviaba los medicamentos: prorrump\u00eda en amenazas, que terminaban en gemidos: esforz\u00e1base la ira, y declinaba en pusilanimidad: la persuasi\u00f3n le ofend\u00eda, y los consuelos le irritaban: cobr\u00f3 el sentido para perder el entendimiento; y pareci\u00f3 conveniente dejarle por un rato, y dar alg\u00fan tiempo a la consideraci\u00f3n, para que se desembarazase de las primeras disonancias de la ofensa. Qued\u00f3 encargado a su familia, y en miserable congoja, batallando con las violencias de su natural, y el abatimiento de su esp\u00edritu, sin aliento para intentar el castigo de los traidores"""

added, removed, modified = diff(prv, mod)

for p,m,context,_,_ in modified:
    print(f"\033[91m{p}\033[0m → \033[94m{m}\033[0m", end='')
    print(f" [{'OCR error' if ocr_error(p,m) else '¿Surface form?'}]", end='')
    printred(f"\t{context}", p)
    print()

contra- tiempos → contratiempos [OCR error]	como uno de los mayores contra - tiempos que se le podian ofrecer
podian → podían [¿Surface form?]	- tiempos que se le podian ofrecer . Hizole retirar á
Hizole → Hízole [¿Surface form?]	se le podian ofrecer . Hizole retirar á su quarto ,
quarto → cuarto [¿Surface form?]	. Hizole retirar á su quarto , y acudió con nueva
ala de- fensa → a la defensa [OCR error]	y acudió con nueva irritación ala de - fensa del quartel ; pero se
quartel → cuartel [¿Surface form?]	ala de - fensa del quartel ; pero se halló sin
pu- dieron → pudieron [OCR error]	á su Rey , ó pu - dieron conocer que iba herido ,
sue len dexar → suelen dejar [¿Surface form?]	, ó fealdad espantosa que sue len dexar en el ánimo al acabarse
quarto de Motezu ma → cuarto de Moctezuma [¿Surface form?]	Pasó luego Hernán Cortés al quarto de Motezu ma , que volvió en si
si → sí [¿Surface form?]	ma , que volvió en si dentro de breve rato ;
fué → fue [¿Surface form?]	impaciente y despechado , qu

## 2. Correct the simple OCR errors found in the whole dataset

Previous:

In [6]:
r['data'][0]["text"]

'La publicacion del Oso se harà dos veces cada se mana, y constará de un pliego en cuarto ; ofreciendo à mas sus redactores, dar los gravados oportunos, siempre que loexija el asuntode que trate. Redactado por un Num. 8. TEMA del Periodico. POLITICA MILITAR. OCTAVA SESION. Abierta la sesion á las dore y un minuto de la noche , 25 de Febrero de 1845 , con asistencia de todos los Señores Representantes, se leyó y aprobó la acta de la Asamblea anterior , ménos en lo tocante à la torre del Convento de Santo Domingo, punto que quedó para ventilarse en mejor ocasion. En seguida se dió cuenta de una nota del Ejecutivo , referente à que urjía la necesidad de organizar un Ejército ; pues decia el Excmo. Decano: - "Un poder sin bayonetas vale tanto como un cero puesto á la izquierda."'

In [7]:
it=0
while True:
    ocrcount = 0
    errors = []

    for i,d in enumerate(r['data']):
        prv = d["text"]
        mod = d["resp"]
        try:
            added, removed, modified = diff(prv, mod)
        except AssertionError as e:
            print(f"Error at {i}:\n{prv}\n{mod}")
            raise e

        er = list()
        for p,m,context,idx1,idx2 in modified:
            tup = (p,m)
            isocr = ocr_error(p,m)
            if isocr:
                er.append({"prv":p, "mod":m, "idx1": idx1, "idx2":idx2, "ctx":context})
                ocrcount += 1
        errors.append(er)

    if ocrcount == 0: break
    assert len(errors) == len(df)

    for i,e in enumerate(errors):
        text = r['data'][i]['text']
        chdif = 0
        for v in e:
            p = v['prv']
            m = v['mod']
            ctx = v['ctx']
            idx1 = v['idx1'] - chdif
            idx2 = v['idx2'] - chdif
            text = text[:idx1] + m + text[idx2:]
            chdif += len(p) - len(m)
        df.loc[i, "text"] = text
        r['data'][i]['text'] = text
    
    it+=1
    print(f"Iteration {it} completed! ({ocrcount} OCR errors found)")

Iteration 1 completed! (37238 OCR errors found)
Iteration 2 completed! (183 OCR errors found)
Iteration 3 completed! (2 OCR errors found)


After update of only OCR non-letter errors:

In [8]:
r['data'][0]["text"]

'La publicacion del Oso se harà dos veces cada semana, y constará de un pliego en cuarto ; ofreciendo à mas sus redactores, dar los gravados oportunos, siempre que lo exija el asunto de que trate. Redactado por un Num. 8. TEMA del Periodico. POLITICA MILITAR. OCTAVA SESION. Abierta la sesion á las dore y un minuto de la noche , 25 de Febrero de 1845 , con asistencia de todos los Señores Representantes, se leyó y aprobó la acta de la Asamblea anterior , ménos en lo tocante à la torre del Convento de Santo Domingo, punto que quedó para ventilarse en mejor ocasion. Enseguida se dió cuenta de una nota del Ejecutivo , referente à que urjía la necesidad de organizar un Ejército ; pues decia el Excmo. Decano: - "Un poder sin bayonetas vale tanto como un cero puesto á la izquierda."'

Save a first version of the corrected dataset:

In [10]:
df.to_csv("../data/corrected-latam-xix.tsv", sep="\t", index=False)
df.to_parquet('../data/corrected-latam-xix.parquet')

## 3. Save the rest of the corrections to classify

The rest of the corrections can be classified between:
- An OCR error with letter errors
- Surface forms
- None of the above (LLM hallucinations)

In [11]:
fixes = dict()

for i,d in enumerate(r['data']):
    prv = d["text"]
    mod = d["resp"]
    try:
        added, removed, modified = diff(prv, mod)
    except AssertionError as e:
        print(f"Error at {i}:\n{prv}\n{mod}")
        raise e

    for p,m,context,idx1,idx2 in modified:
        tup = (p,m)
        assert not ocr_error(p,m), f"ERROR at {i}. OCR error detected: {p} -> {m} ({context})"
        fixes[tup] = fixes.get(tup, {'usages':[], 'freq':0})
        fixes[tup]['usages'].append((i,idx1,idx2,context))
        fixes[tup]['freq'] += 1

fixes = dict(sorted(fixes.items(), key=lambda x: x[1]['freq'], reverse=True))

fixes_list = []
for k,v in fixes.items():
    v['change'] = list(k)
    fixes_list.append(v)

In [12]:
fixes_list[0]['change'], fixes_list[0]['freq']

(['á', 'a'], 31606)

Save the corrections to a file:

In [13]:
with open(CORRECTIONS_FILE, 'w') as outfile:
    json.dump(fixes_list, outfile, indent=2)